In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pyntcloud import PyntCloud
import torch.nn.functional as F
import open3d as o3d

In [ ]:
class VoxelDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return torch.from_numpy(sample).float()

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=3, padding=1),   # 113x113x113 -> 113x113x113
            nn.ReLU(True),
            nn.MaxPool3d(2, stride=2),                   # 113x113x113 -> 56x56x56
            nn.Conv3d(32, 64, kernel_size=3, padding=1),  # 56x56x56 -> 56x56x56
            nn.ReLU(True),
            nn.MaxPool3d(2, stride=2),                   # 56x56x56 -> 28x28x28
            nn.Conv3d(64, 128, kernel_size=3, padding=1), # 28x28x28 -> 28x28x28
            nn.ReLU(True),
            nn.MaxPool3d(2, stride=2)                    # 28x28x28 -> 14x14x14
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=3, stride=2), # 14x14x14 -> 28x28x28
            nn.ReLU(True),
            nn.ConvTranspose3d(64, 32, kernel_size=3, stride=2),  # 28x28x28 -> 56x56x56
            nn.ReLU(True),
            nn.ConvTranspose3d(32, 1, kernel_size=3, stride=2),   # 56x56x56 -> 112x112x112
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
directory = "abc-dataset-ply/"
files = sorted([filename for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))])
dataset = []

def convert_to_binvox(path):
    point_cloud = np.loadtxt(path, skiprows=12)[:, 0:3]
    df = pd.DataFrame(data=point_cloud, columns=['x','y','z'])
    cloud = PyntCloud(df)
    voxelgrid_id = cloud.add_structure("voxelgrid", n_x=119, n_y=119, n_z=119)
    voxelgrid = cloud.structures[voxelgrid_id]
    Binary_voxel_array = voxelgrid.get_feature_vector(mode="binary")
    dataset.append(Binary_voxel_array)

for i in files:
    path = os.path.join(directory, i)
    convert_to_binvox(path)

In [ ]:
train_data = dataset[:100]
test_data = dataset[100:]

# Create PyTorch datasets and dataloaders
train_dataset = VoxelDataset(train_data)
test_dataset = VoxelDataset(test_data)

# Determine the device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the model, optimizer, and loss function
autoencoder = Autoencoder().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=3, shuffle=False)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
num_epochs = 7
for epoch in range(num_epochs):
    for data in train_loader:
        optimizer.zero_grad()
        data = data.to(device)
        outputs = autoencoder(data.unsqueeze(1).to(device))
        loss = criterion(outputs, data.unsqueeze(1).to(device))
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')